In [3]:
import os
import io
import math
import random
import requests
import numpy as np
import pandas as pd
from subprocess import run
import multiprocessing as mp

from typing import List, Dict, Any, Tuple
from tqdm import tqdm_notebook as tqdm

random.seed(42)

In [4]:
# Inputs

cluster_name = "cluster2_16TB"
date = "20240119"
start_idx = 0
num_traces_to_include = 100
url_template = "https://storage.googleapis.com/thesios-io-traces/%s/%s/%s"
initialize_data_dir = False
remove_zero_size_ops = True
data_file_from_idx = lambda idx: "data-00%s-of-00100" % str(idx).zfill(3)
app_to_keep = "3f524cdc07a11d7c6220bdb049fe8dd41b27483c96cc59b581e022d547290d69"
num_ops_to_keep = 700000
trace_folder = "traces"

# Computed
def format_app_to_keep(s: str) -> str:
    if len(s) < 12:
        return s
    return s[:8]

assert start_idx >= 0 and start_idx <= 100, "Invalid start_idx: %d" % start_idx
end_idx = start_idx + num_traces_to_include - 1
assert end_idx >= 0 and end_idx <= 100, "Invalid end_idx: %d" % end_idx
output_trace_file = "trace_%s_%s" % (cluster_name, date)
if app_to_keep:
    output_trace_file += "_app_%s" % format_app_to_keep(app_to_keep)
output_trace_file += ".txt"
data_dir = "data_%s_%s" % (cluster_name, date)
if app_to_keep:
    data_dir += "_app_%s" % format_app_to_keep(app_to_keep)
data_dir += "_temp"


In [5]:
if not os.path.exists(trace_folder):
    os.makedirs(trace_folder, exist_ok=True)

In [6]:
# Helper functions

def download_to_file_single_arg(args):
    return download_to_file(*args)

def download_to_file(url, filename):
    response = requests.get(url)
    with open(filename, "wb") as f:
        f.write(response.content)

def format_bytes(bytes: int) -> str:
    for unit in ["B", "KB", "MB", "GB", "TB"]:
        if bytes < 1024:
            return f"{bytes:.2f} {unit}"
        bytes /= 1024
    return f"{bytes:.2f} PB"

def process_trace(trace) -> Dict[str, Dict[str, int]]:
    # We want to process the trace, operation by operation.
    # Let's simplify: assume everything pre-exists, and we only need to keep track of the last read/write offset for each file.

    # We will keep track of the last read offset for each file.
    # Initialize dictionaries to store file metadata
    file_metadata = {}

    for _, row in trace.iterrows():
        filename = row["filename"]
        offset = row["file_offset"]
        io_size = row["request_io_size_bytes"]
        operation = row["op_type"]

        if filename not in file_metadata:
            file_metadata[filename] = {
                "max_read_offset": 0,
                "max_write_offset": 0,
                "max_initialized_offset": 0,
            }

        metadata = file_metadata[filename]

        if operation == "READ":
            metadata["max_initialized_offset"] = max(metadata["max_initialized_offset"], offset + io_size)
            metadata["max_read_offset"] = max(metadata["max_read_offset"], offset + io_size)
        elif operation == "WRITE":
            metadata["max_initialized_offset"] = max(metadata["max_initialized_offset"], offset + io_size)
            metadata["max_write_offset"] = max(metadata["max_write_offset"], offset + io_size)

    return file_metadata

def print_file_metadata_stats(file_metadata) -> Tuple[int, int, int]:
    total_size_initialized = sum([metadata["max_initialized_offset"] for metadata in file_metadata.values()])
    total_size_read = sum([metadata["max_read_offset"] for metadata in file_metadata.values()])
    total_size_written = sum([metadata["max_write_offset"] for metadata in file_metadata.values()])

    print(f"Total size initialized: {format_bytes(total_size_initialized)}")
    print(f"Total size read: {format_bytes(total_size_read)}")
    print(f"Total size written: {format_bytes(total_size_written)}")
    return total_size_initialized, total_size_read, total_size_written

# Initialize a new directory and write the files needed to run the trace
def initialize_file(file_path: str, size: int):
    """Initialize a file with pseudo-random data."""
    with open(file_path, "wb") as f:
        # Write data in chunks to avoid excessive memory usage
        chunk_size = 16 * 1024 * 1024  # 16 MB chunks
        remaining_size = size

        while remaining_size > 0:
            write_size = min(chunk_size, remaining_size)
            data = np.random.bytes(write_size)
            f.write(data)
            remaining_size -= write_size
    # print(f"Initialized file: {file_path} with size: {format_bytes(size)}")

def initialize_file_single_arg(args):
    return initialize_file(*args)

def create_trace_data_dir(data_dir: str, file_metadata: Dict[str, Dict[str, int]]):
    print("Initializing data directory: %s" % data_dir)
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    else:
        print("Data directory already exists, not doing anything.")
        return

    # Process files in parallel
    total_files = len(file_metadata)
    print(f"Initializing {total_files} files...")

    with mp.Pool(mp.cpu_count()) as pool:
        tasks = []
        for filename, metadata in file_metadata.items():
            file_path = os.path.join(data_dir, filename)
            size = metadata["max_initialized_offset"]
            tasks.append((file_path, size))

        # Use multiprocessing to parallelize file initialization
        for _ in tqdm(pool.imap(initialize_file_single_arg, tasks), total=total_files):
            pass

    print("File initialization complete.")



In [7]:
local_files = []
download_tasks = []
for idx in range(start_idx, end_idx + 1):
    data_file = data_file_from_idx(idx)
    local_file = "%s_%s_%s.csv" % (cluster_name, date, data_file)
    local_file = os.path.join(trace_folder, local_file)
    download_url = url_template % (cluster_name, date, data_file)
    local_files.append(local_file)

    if not os.path.exists(local_file):
        print("Downloading %s to %s" % (download_url, local_file))
        download_tasks.append((download_url, local_file))

if len(download_tasks) > 0:
    with mp.Pool(mp.cpu_count()) as pool:
        for _ in tqdm(pool.imap(download_to_file_single_arg, download_tasks), total=len(download_tasks)):
            pass

# Combine all the traces into a single file
trace_dfs = [ pd.read_csv(local_file) for local_file in local_files ]
combined_trace = pd.concat(trace_dfs, ignore_index=True)
trace = combined_trace


In [8]:
trace.head()

,filename,file_offset,application,c_time,io_zone,redundancy_type,op_type,service_class,from_flash_cache,cache_hit,request_io_size_bytes,disk_io_size_bytes,response_io_size_bytes,start_time,disk_time,simulated_disk_start_time,simulated_latency
0,5def9de616cdd1939104e38cc9feb4f049ec7fad38ab6c...,4202496,08d81b9cb3e4dcc7e55af443437cf6bf7be30b970a31bf...,1705651199,WARM,ERASURE_CODED,WRITE,OTHER,0,-1,0,0,0,1.705651e+09,0.00000,0.000000e+00,0.000043
1,5def9de616cdd1939104e38cc9feb4f049ec7fad38ab6c...,4202496,08d81b9cb3e4dcc7e55af443437cf6bf7be30b970a31bf...,1705651199,WARM,ERASURE_CODED,WRITE,OTHER,0,-1,1050624,1050624,0,1.705651e+09,0.00000,0.000000e+00,0.000433
2,e9eb9de6ce9e393bf9e7a69cdfb8fd2bb1b7891626b142...,1050624,1c0fab5e9c61ea6f0fd93b1ef13442b30914bd61b7c246...,1705649755,WARM,ERASURE_CODED,READ,OTHER,0,0,430080,1056768,430080,1.705651e+09,0.01718,1.705651e+09,0.017180
3,0ea93d8ddb4672ccf2a422ec3297b85f66b534fef60101...,6093361,c994007e4688d5bb2a549cdcf077c20baa9da6d949e893...,1705629496,WARM,REPLICATED,WRITE,LATENCY_SENSITIVE,0,-1,0,0,0,1.705651e+09,0.00000,0.000000e+00,0.000086
4,0ea93d8ddb4672ccf2a422ec3297b85f66b534fef60101...,6093361,c994007e4688d5bb2a549cdcf077c20baa9da6d949e893...,1705629496,WARM,REPLICATED,WRITE,LATENCY_SENSITIVE,0,-1,363018,363018,0,1.705651e+09,0.00000,0.000000e+00,0.010792


In [9]:
# Print unique applications
print("Number of unique applications: %d" % len(trace["application"].unique()))

# Print number of operations per application
operations_per_application = trace.groupby("application").size().sort_values(ascending=False)
print("Number of operations per application:")
print(operations_per_application)

Number of unique applications: 849
Number of operations per application:
application
c994007e4688d5bb2a549cdcf077c20baa9da6d949e893173f642c66047e3c3c    3451429
3f524cdc07a11d7c6220bdb049fe8dd41b27483c96cc59b581e022d547290d69     867452
7e9de8e118680d9d454a63c405a6ba56475db9f57e27f3fcef6f722cc8b6d406     601510
93ad3bf6273c13fcaa8f8b1793a666f22aa75d976b099cdfdacacc0ccb6c64ff     470144
89bbe87e189233e198c0913677ecb8eb6bd0e6c651e3ba236e15863428894553     305636
                                                                     ...   
3262ffeffa1f934ded2b1c7455c1946e2bdf9c4fcb1daa1e9e2995f6f77e12c2          1
f7e2f5a011c70d41ce6ee8654a65f0145b51e0f28d9b7f02b490d064f7263b55          1
ee0e30dce552021557ad0456c79474ead4c35f1f2a959d6cb0a617b8e5c4aac6          1
f6aadc7f500075f82b060507e4bef6f01b5b073b6e39dbcfaa340a82912edc6e          1
88981a098017224b43c68cbee1067e95f52ce2d779de254f92687006fcb64066          1
Length: 849, dtype: int64


In [10]:
if remove_zero_size_ops:
    trace = trace[trace["request_io_size_bytes"] > 0].reset_index()
if app_to_keep and len(app_to_keep) > 0:
    trace = trace[trace["application"] == app_to_keep].head(num_ops_to_keep).reset_index(drop=True).copy()

In [11]:
# How many lines is the trace?
print("Number of lines in trace: %d" % len(trace))

# What is the proportion of read and write operations?
read_ops = len(trace[trace["op_type"] == "READ"])
write_ops = len(trace[trace["op_type"] == "WRITE"])
print("Read operations: %d" % read_ops)
print("Write operations: %d" % write_ops)
print("Read proportion: %.2f%%" % (100 * read_ops / (read_ops + write_ops)))

read_bytes = trace[trace["op_type"] == "READ"]["request_io_size_bytes"].sum()
write_bytes = trace[trace["op_type"] == "WRITE"]["request_io_size_bytes"].sum()
total_bytes = read_bytes + write_bytes
print("Read bytes: %s" % format_bytes(read_bytes))
print("Write bytes: %s" % format_bytes(write_bytes))
print("Total bytes: %s" % format_bytes(total_bytes))

Number of lines in trace: 583961
Read operations: 258466
Write operations: 325495
Read proportion: 44.26%
Read bytes: 249.38 GB
Write bytes: 228.60 GB
Total bytes: 477.98 GB


In [12]:
num_unique_filenames = trace["filename"].nunique()
print(f"Number of unique filenames: {num_unique_filenames}")
print("\nUnique filenames:")
print(trace["filename"].unique())
if num_unique_filenames > 100000:
    raise Exception("Too many unique filenames: %d" % num_unique_filenames)

Number of unique filenames: 30027

Unique filenames:
['bc51c4adbb722097b5d5d6740e50a55a0302b453623dd364cb0ca35aa410c7a2'
 '334d52f28ddcc13458079787436d6586543a6a45a4ba242a0a73cf46ab776d2f'
 'a143529558261b9f95d760d62bc753ef85954cb52a2b9a6597d28ee7ab179c31' ...
 '5e76cd9798302b3fc243a5ba1c932c244f4272cccd3324eb99894ab4cc2f4566'
 'de0d256c816ed204956c4eea58e0284fa9cfc49cb930ddd26c4100d7b344d418'
 '327cabe75cb37bcb166fe41568ba4d028e88aae6def7b202420c6389ce6a4599']


In [13]:
file_metadata = process_trace(trace)
total_size_initialized, total_size_read, total_size_written = print_file_metadata_stats(file_metadata)

Total size initialized: 230.22 GB
Total size read: 195.79 GB
Total size written: 228.61 GB


In [14]:
GiB = 2 ** 30
if total_size_initialized > 350 * GiB:
    raise Exception("Too much data initialized: %s" % format_bytes(total_size_initialized))

In [15]:
if initialize_data_dir:
    create_trace_data_dir(data_dir, file_metadata)

In [16]:
trace.head()

,index,filename,file_offset,application,c_time,io_zone,redundancy_type,op_type,service_class,from_flash_cache,cache_hit,request_io_size_bytes,disk_io_size_bytes,response_io_size_bytes,start_time,disk_time,simulated_disk_start_time,simulated_latency
0,49,bc51c4adbb722097b5d5d6740e50a55a0302b453623dd3...,5790005,3f524cdc07a11d7c6220bdb049fe8dd41b27483c96cc59...,1705629339,WARM,REPLICATED,WRITE,OTHER,0,-1,16015,16015,0,1.705651e+09,0.000000,0.000000e+00,0.000283
1,127,334d52f28ddcc13458079787436d6586543a6a45a4ba24...,29417472,3f524cdc07a11d7c6220bdb049fe8dd41b27483c96cc59...,1705648549,WARM,REPLICATED,READ,THROUGHPUT_ORIENTED,0,0,2101248,2101248,2101248,1.705651e+09,0.009916,1.705651e+09,0.089438
2,152,bc51c4adbb722097b5d5d6740e50a55a0302b453623dd3...,5806020,3f524cdc07a11d7c6220bdb049fe8dd41b27483c96cc59...,1705629339,WARM,REPLICATED,WRITE,OTHER,0,-1,13338,13338,0,1.705651e+09,0.000000,0.000000e+00,0.000071
3,225,bc51c4adbb722097b5d5d6740e50a55a0302b453623dd3...,5819358,3f524cdc07a11d7c6220bdb049fe8dd41b27483c96cc59...,1705629339,WARM,REPLICATED,WRITE,OTHER,0,-1,9720,9720,0,1.705651e+09,0.000000,0.000000e+00,0.000073
4,231,334d52f28ddcc13458079787436d6586543a6a45a4ba24...,31518720,3f524cdc07a11d7c6220bdb049fe8dd41b27483c96cc59...,1705648549,WARM,REPLICATED,READ,THROUGHPUT_ORIENTED,0,0,2101248,2105344,2101248,1.705651e+09,0.024331,1.705651e+09,0.024331


In [17]:
# Format the trace file to be used in the benchmark client
# The expected format is:
# <op_type> <filename>:<offset>:<size>
# Example:
# READ file1:0:4096
# WRITE file2:0:4096

def convert_trace_to_bench_client_format(trace: pd.DataFrame, output_file: str):
    with open(output_file, "w") as f:
        for _, row in trace.iterrows():
            filename = row["filename"]
            offset = row["file_offset"]
            io_size = row["request_io_size_bytes"]
            operation = row["op_type"]

            f.write(f"{operation} {filename}:{offset}:{io_size}\n")

if not os.path.exists(output_trace_file):
    print("Converting trace to benchmark client format...")
    convert_trace_to_bench_client_format(trace, output_trace_file)
    print("Trace file converted to benchmark client format: %s" % output_trace_file)
else:
    print("Trace file already exists: %s" % output_trace_file)
    print("Not overwriting.")

Converting trace to benchmark client format...
Trace file converted to benchmark client format: trace_cluster2_16TB_20240119_app_3f524cdc.txt


In [18]:
PAGE_SIZE = 4096

def convert_trace_to_libcachesim_format(trace: pd.DataFrame, output_file: str):

    with open(output_file, "w") as f:
        f.write("time,obj_id,obj_size\n")
        for _, row in trace.iterrows():
            filename = row["filename"]
            offset = row["file_offset"]
            timestamp = row["start_time"]
            io_size = row["request_io_size_bytes"]
            start_page_num = offset // PAGE_SIZE
            end_page_num = math.ceil((offset + io_size) / PAGE_SIZE)
            for page_num in range(start_page_num, end_page_num):
                offset = page_num * PAGE_SIZE
                io_size = PAGE_SIZE
                obj_id = f"{filename}:{offset}"
                f.write(f"{timestamp},{obj_id},{io_size}\n")

prefix = output_trace_file
if prefix.endswith(".txt"):
    prefix = prefix[:-4]
output_trace_file_libcachesim = prefix + "_libcachesim.csv"
if not os.path.exists(output_trace_file_libcachesim):
    print("Converting trace to libcachesim format...")
    convert_trace_to_libcachesim_format(trace, output_trace_file_libcachesim)
    print("trace file converted to libcachesim format: %s" % output_trace_file_libcachesim)
else:
    print("libcachesim trace file already exists: %s" % output_trace_file_libcachesim)
    print("Not overwriting.")


output_trace_file_libcachesim_binary = output_trace_file_libcachesim + ".oracleGeneral"
if not os.path.exists(output_trace_file_libcachesim_binary):
    print("Converting trace to libcachesim binary format...")
    trace_conv_bin = "/mydata/libCacheSim/_build/bin/traceConv"
    if not os.path.exists(trace_conv_bin):
        raise Exception("traceConv binary does not exist: %s" % trace_conv_bin)
    csv_opts = "time-col=1, obj-id-col=2, obj-size-col=3, delimiter=,, has-header=true"
    cmd = [trace_conv_bin, output_trace_file_libcachesim, "csv", "-t", csv_opts]
    run(cmd)
    print("trace file converted to libcachesim binary format: %s" % output_trace_file_libcachesim_binary)
else:
    print("libcachesim bninary trace file already exists: %s" % output_trace_file_libcachesim_binary)


Converting trace to libcachesim format...
trace file converted to libcachesim format: trace_cluster2_16TB_20240119_app_3f524cdc_libcachesim.csv
Converting trace to libcachesim binary format...


[INFO]  11-08-2024 14:39:55 cli_parser.cpp:231  (tid=139864297411904): trace path: trace_cluster2_16TB_20240119_app_3f524cdc_libcachesim.csv, trace_type CSV_TRACE, trace type params: time-col=1, obj-id-col=2, obj-size-col=3, delimiter=,, has-header=true
[INFO]  11-08-2024 14:45:53 traceConv.cpp:63   (tid=139864297411904): trace_cluster2_16TB_20240119_app_3f524cdc_libcachesim.csv: 125.64 M requests in total
[INFO]  11-08-2024 15:02:45 traceConv.cpp:95   (tid=139864297411904): trace_cluster2_16TB_20240119_app_3f524cdc_libcachesim.csv: 100 M requests (381.47 GB), trace time 67637, working set 47656072 object, 195199270912 B (181.79 GB)
[INFO]  11-08-2024 15:07:03 traceConv.cpp:122  (tid=139864297411904): trace_cluster2_16TB_20240119_app_3f524cdc_libcachesim.csv: 125 M requests (479.29 GB), trace time -94148964451233, working set 60320626 object, 247073284096 B (230.10 GB), reversing output...
[INFO]  11-08-2024 15:07:09 traceConv.cpp:246  (tid=139864297411904): trace_cluster2_16TB_2024011

trace file converted to libcachesim binary format: trace_cluster2_16TB_20240119_app_3f524cdc_libcachesim.csv.oracleGeneral


In [19]:
# Find lines where request_io_size_bytes != disk_io_size_bytes
mismatched_io_sizes = trace[trace["request_io_size_bytes"] != trace["disk_io_size_bytes"]]

# Display the first few rows of mismatched IO sizes
print("Rows where request_io_size_bytes != disk_io_size_bytes:")
mismatched_io_sizes.head()

Rows where request_io_size_bytes != disk_io_size_bytes:


,index,filename,file_offset,application,c_time,io_zone,redundancy_type,op_type,service_class,from_flash_cache,cache_hit,request_io_size_bytes,disk_io_size_bytes,response_io_size_bytes,start_time,disk_time,simulated_disk_start_time,simulated_latency
4,231,334d52f28ddcc13458079787436d6586543a6a45a4ba24...,31518720,3f524cdc07a11d7c6220bdb049fe8dd41b27483c96cc59...,1705648549,WARM,REPLICATED,READ,THROUGHPUT_ORIENTED,0,0,2101248,2105344,2101248,1.705651e+09,0.024331,1.705651e+09,0.024331
8,386,334d52f28ddcc13458079787436d6586543a6a45a4ba24...,35721216,3f524cdc07a11d7c6220bdb049fe8dd41b27483c96cc59...,1705648549,WARM,REPLICATED,READ,THROUGHPUT_ORIENTED,0,0,2101248,2105344,2101248,1.705651e+09,0.022177,1.705651e+09,0.022219
9,405,a143529558261b9f95d760d62bc753ef85954cb52a2b9a...,1050624,3f524cdc07a11d7c6220bdb049fe8dd41b27483c96cc59...,1705650838,WARM,ERASURE_CODED,READ,OTHER,0,0,595968,602112,595968,1.705651e+09,0.014296,1.705651e+09,0.022258
11,423,a143529558261b9f95d760d62bc753ef85954cb52a2b9a...,1622016,3f524cdc07a11d7c6220bdb049fe8dd41b27483c96cc59...,1705650838,WARM,ERASURE_CODED,READ,OTHER,0,0,479232,454656,479232,1.705651e+09,0.001657,1.705651e+09,0.003175
15,631,334d52f28ddcc13458079787436d6586543a6a45a4ba24...,39923712,3f524cdc07a11d7c6220bdb049fe8dd41b27483c96cc59...,1705648549,WARM,REPLICATED,READ,THROUGHPUT_ORIENTED,0,0,2101248,2105344,2101248,1.705651e+09,0.007642,1.705651e+09,0.007841
